In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 

In [ ]:
data='In the town of Athy one Jeremy Lanigan \n Battered away til he hadnt a pound. \nHis father died and made him a man again \n Left him a farm and ten acres of ground. \nHe gave a grand party for friends and relations \nWho didnt forget him when come to the wall, \nAnd if youll but listen Ill make your eyes glisten \nOf the rows and the ructions of Lanigans Ball. \nMyself to be sure got free invitation, \nFor all the nice girls and boys I might ask, \nAnd just in a minute both friends and relations \nWere dancing round merry as bees round a cask. \nJudy ODaly, that nice little milliner, \nShe tipped me a wink for to give her a call, \nAnd I soon arrived with Peggy McGilligan \nJust in time for Lanigans Ball. \nThere were lashings of punch and wine for the ladies, \nPotatoes and cakes; there was bacon and tea, \nThere were the Nolans, Dolans, OGradys \nCourting the girls and dancing away. \nSongs they went round as plenty as water, \nThe harp that once sounded in Taras old hall,\nSweet Nelly Gray and The Rat Catchers Daughter,\nAll singing together at Lanigans Ball. \nThey were doing all kinds of nonsensical polkas \nAll round the room in a whirligig. \nJulia and I, we banished their nonsense \nAnd tipped them the twist of a reel and a jig. \nAch mavrone, how the girls got all mad at me \nDanced til youd think the ceiling would fall. \nFor I spent three weeks at Brooks Academy \nLearning new steps for Lanigans Ball. \nThree long weeks I spent up in Dublin, \nThree long weeks to learn nothing at all,\n Three long weeks I spent up in Dublin, \nLearning new steps for Lanigans Ball. \nShe stepped out and I stepped in again, \nI stepped out and she stepped in again, \nShe stepped out and I stepped in again, \nLearning new steps for Lanigans Ball. \nBoys were all merry and the girls they were hearty \nAnd danced all around in couples and groups, \nTil an accident happened, young Terrance McCarthy \nPut his right leg through miss Finnertys hoops. \nPoor creature fainted and cried Meelia murther, \nCalled for her brothers and gathered them all. \nCarmody swore that hed go no further \nTil he had satisfaction at Lanigans Ball. \nIn the midst of the row miss Kerrigan fainted, \nHer cheeks at the same time as red as a rose. \nSome of the lads declared she was painted, \nShe took a small drop too much, I suppose. \nHer sweetheart, Ned Morgan, so powerful and able, \nWhen he saw his fair colleen stretched out by the wall, \nTore the left leg from under the table \nAnd smashed all the Chaneys at Lanigans Ball. \nBoys, oh boys, twas then there were runctions. \nMyself got a lick from big Phelim McHugh. \nI soon replied to his introduction \nAnd kicked up a terrible hullabaloo. \nOld Casey, the piper, was near being strangled. \nThey squeezed up his pipes, bellows, chanters and all. \nThe girls, in their ribbons, they got all entangled \nAnd that put an end to Lanigans Ball.'
corpus=data.lower().split('\n')

In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(corpus)
vocab_size=len(tokenizer.word_index) + 1


In [ ]:
input_sentences_tokens=[]
for item in corpus:
  token_list=tokenizer.texts_to_sequences([item])[0]
  for i in range(1,len(token_list)):
    sub_sentence_token=token_list[:i+1]
    input_sentences_tokens.append(sub_sentence_token)


In [ ]:
max_sequence_len=max([len(x) for x in input_sentences_tokens])
padded_sequences=pad_sequences(input_sentences_tokens,maxlen=max_sequence_len,padding='pre')


In [ ]:
padded_sequences

array([[  0,   0,   0, ...,   0,   4,   2],
       [  0,   0,   0, ...,   4,   2,  66],
       [  0,   0,   0, ...,   2,  66,   8],
       ...,
       [  0,   0,   0, ...,  60, 262,  13],
       [  0,   0,   0, ..., 262,  13,   9],
       [  0,   0,   0, ...,  13,   9,  10]], dtype=int32)

In [ ]:
features_x=padded_sequences[:,:-1]
labels_y=padded_sequences[:,-1]



In [ ]:
from tensorflow.keras.utils import to_categorical
encoded_labels_y=to_categorical(labels_y,num_classes=vocab_size)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,Bidirectional


model=Sequential()

model.add(Embedding(vocab_size,64,input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(vocab_size,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(features_x,encoded_labels_y,epochs=500)

Epoch 1/500
15/15 [==============================] - 0s 10ms/step - loss: 5.5676 - accuracy: 0.0199
Epoch 2/500
15/15 [==============================] - 0s 9ms/step - loss: 5.5330 - accuracy: 0.0508
Epoch 3/500
15/15 [==============================] - 0s 10ms/step - loss: 5.4218 - accuracy: 0.0508
Epoch 4/500
15/15 [==============================] - 0s 10ms/step - loss: 5.1875 - accuracy: 0.0508
Epoch 5/500
15/15 [==============================] - 0s 9ms/step - loss: 5.0625 - accuracy: 0.0464
Epoch 6/500
15/15 [==============================] - 0s 9ms/step - loss: 5.0172 - accuracy: 0.0508
Epoch 7/500
15/15 [==============================] - 0s 9ms/step - loss: 4.9835 - accuracy: 0.0530
Epoch 8/500
15/15 [==============================] - 0s 9ms/step - loss: 4.9359 - accuracy: 0.0552
Epoch 9/500
15/15 [==============================] - 0s 9ms/step - loss: 4.8797 - accuracy: 0.0574
Epoch 10/500
15/15 [==============================] - 0s 9ms/step - loss: 4.8133 - accuracy: 0.0574
Epoch 

In [ ]:
sample_sentence='Laurence went to Dublin'
predict_next_words=100

for _ in range(predict_next_words):
  sample_token=tokenizer.texts_to_sequences([sample_sentence])[0]
  sample_padded_token=pad_sequences([sample_token],padding='pre',maxlen=max_sequence_len-1)
  prediction=model.predict_classes(sample_padded_token)

  output_word=''
  for word,index in tokenizer.word_index.items():
    if index==prediction:
      output_word=word
      break

  sample_sentence=sample_sentence + ' ' + output_word


print(sample_sentence)


  



Laurence went to Dublin round the miss kerrigan fainted mavrone the that dolans hoops call ask ask hearty girls how ball ball all relations relations hearty hearty ladies ladies boys mavrone the girls in further wall and catchers and bacon tea cask tea glisten creature we hoops hoops hall strangled up stepped gray the again again girls by wall the groups catchers ask lanigan glisten glisten creature we hoops hoops call once hall glisten saw weeks steps ball steps steps water steps water water water water water water water water water water we hoops hall call glisten that hall hall hall hall mccarthy glisten
